# Author: Justin Hsi
## Part  1 of cleaning lending club payment history

In [1]:
import dir_constants as dc
from tqdm import tqdm_notebook

In [2]:
PATH = '/home/justin/rsync_dl_rig/unzipped_lc_csvs/PMTHIST_ALL_201808.csv'

In [3]:
# Set some constants __________________________________________________________
platform = 'lendingclub'

# Set data_path _______________________________________________________________
data_path = dc.home_path + '/rsync_dl_rig/unzipped_lc_csvs/'

# Load in pmt_hist_file _______________________________________________________
files = os.listdir(data_path)
# loan_hist_filename = [file_ for file_ in files if file_.startswith('PMTHIST')][0]
# pmt_hist = pd.read_csv(data_path + '{0}'.format(loan_hist_filename), low_memory=False)
pmt_hist = pd.read_csv(PATH, low_memory=False)
print("{:,}".format(len(pmt_hist)) + " rows of pmt_hist loaded")

# Set loan ids as strings _____________________________________________________
pmt_hist['LOAN_ID'] = pmt_hist['LOAN_ID'].astype(int)
print('payment history for', len(pmt_hist['LOAN_ID'].unique()), 'different loan ids')

# Round values to 3 decimal places ____________________________________________
pmt_hist = pmt_hist.round(3)

# renaming columns ____________________________________________________________
rename_col_dict = {
    'LOAN_ID': 'loan_id',
    'PBAL_BEG_PERIOD': 'outs_princp_beg',
    'PRNCP_PAID': 'princp_paid',
    'INT_PAID': 'int_paid',
    'FEE_PAID': 'fee_paid',
    'DUE_AMT': 'amt_due',
    'RECEIVED_AMT': 'amt_paid',
    'RECEIVED_D': 'pmt_date',
    'PERIOD_END_LSTAT': 'status_period_end',
    'MONTH': 'date',
    'PBAL_END_PERIOD': 'outs_princp_end',
    'MOB': 'm_on_books',
    'CO': 'charged_off_this_month',
    'COAMT': 'charged_off_amt',
    'InterestRate': 'int_rate',
    'IssuedDate': 'issue_d',
    'MONTHLYCONTRACTAMT': 'monthly_pmt',
    'dti': 'dti',
    'State': 'addr_state',
    'HomeOwnership': 'home_ownership',
    'MonthlyIncome': 'm_income',
    'EarliestCREDITLine': 'first_credit_line',
    'OpenCREDITLines': 'open_credit_lines',
    'TotalCREDITLines': 'total_credit_lines',
    'RevolvingCREDITBalance': 'revol_credit_bal',
    'RevolvingLineUtilization': 'revol_line_util',
    'Inquiries6M': 'inq_6m',
    'DQ2yrs': 'dq_24m',
    'MonthsSinceDQ': 'm_since_dq',
    'PublicRec': 'public_recs',
    'MonthsSinceLastRec': 'm_since_rec',
    'EmploymentLength': 'emp_len',
    'currentpolicy': 'current_policy',
    'grade': 'grade',
    'term': 'term',
    'APPL_FICO_BAND': 'fico_apply',
    'Last_FICO_BAND': 'fico_last',
    'VINTAGE': 'vintage',
    'PCO_RECOVERY': 'recovs',
    'PCO_COLLECTION_FEE': 'recov_fees',
}

pmt_hist.rename(columns=rename_col_dict, inplace=True)

37,327,674 rows of pmt_hist loaded
payment history for 2003523 different loan ids


# try and reduces memory

In [4]:
import j_utils.munging

In [5]:
changed_cols, pmt_hist = j_utils.munging.compress_memory(pmt_hist)
pmt_hist.info(memory_usage='deep')

100%|██████████| 26/26 [00:13<00:00,  1.87it/s]


changed dtypes of 26 cols
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37327674 entries, 0 to 37327673
Data columns (total 40 columns):
pmt_date                  object
status_period_end         object
date                      object
issue_d                   object
addr_state                object
home_ownership            object
first_credit_line         object
inq_6m                    object
public_recs               object
emp_len                   object
grade                     object
fico_apply                object
fico_last                 object
vintage                   object
amt_due                   float32
amt_paid                  float32
charged_off_amt           float32
charged_off_this_month    int8
current_policy            int8
dq_24m                    float32
dti                       float32
fee_paid                  float32
int_paid                  float32
int_rate                  float32
loan_id                   int32
m_income                  float

In [6]:
gc.collect()

263

# Parsing and Cleaning

In [7]:
# There are 5 columns dealing with money: princp_paid, int_paid, fee_paid,
# recovs, and recovs_fee. princp_paid + int_paid + fee_paid is sometimes short
# of amt_paid. Be conservative and rewrite amt_paid to be sum of said 3.
# Also make all_cash_to_inv = amt_paid + recovs - recov_fees
# Fee paid is always positive, and by inspection it is money borrower pays out
pmt_hist[
    'amt_paid'] = pmt_hist['princp_paid'] + pmt_hist['int_paid'] + pmt_hist['fee_paid']
pmt_hist['recovs'].fillna(0, inplace=True)
pmt_hist['recov_fees'].fillna(0, inplace=True)
pmt_hist[
    'all_cash_to_inv'] = pmt_hist['amt_paid'] + pmt_hist['recovs'] - pmt_hist['recov_fees']

# turn all date columns into pandas timestamp _________________________________
month_dict = {
    'jan': '1-',
    'feb': '2-',
    'mar': '3-',
    'apr': '4-',
    'may': '5-',
    'jun': '6-',
    'jul': '7-',
    'aug': '8-',
    'sep': '9-',
    'oct': '10-',
    'nov': '11-',
    'dec': '12-'
}

# pmt_date ____________________________________________________________________
pmt_hist['pmt_date'] = pd.to_datetime(
    pmt_hist['pmt_date'].str[:3].str.lower().replace(month_dict) +
    pmt_hist['pmt_date'].str[3:],
    format='%m-%Y')

# date ________________________________________________________________________
pmt_hist['date'] = pd.to_datetime(
    pmt_hist['date'].str[:3].str.lower().replace(month_dict) +
    pmt_hist['date'].str[3:],
    format='%m-%Y')

# issue_d _____________________________________________________________________
pmt_hist['issue_d'] = pd.to_datetime(
    pmt_hist['issue_d'].str[:3].str.lower().replace(month_dict) +
    pmt_hist['issue_d'].str[3:],
    format='%m-%Y')

# first_credit_line ____________________________________________________________
pmt_hist['first_credit_line'] = pd.to_datetime(
    pmt_hist['first_credit_line'].str[:3].str.lower().replace(month_dict) +
    pmt_hist['first_credit_line'].str[3:],
    format='%m-%Y')

# status_period_end ____________________________________________________________
status_fix = {
    'Current': 'current',
    'Late (31-120 days)': 'late_120',
    'Fully Paid': 'paid',
    'Charged Off': 'charged_off',
    'Default': 'defaulted',
    'Late (16-30 days)': 'late_30',
    'In Grace Period': 'grace_15',
    'Issued': 'current'
}
pmt_hist['status_period_end'] = pmt_hist['status_period_end'].replace(
    status_fix)

# home_ownership _______________________________________________________________
home_ownership_fix = {
    'admin_us': 'other',
    'mortgage': 'mortgage',
    'rent': 'rent',
    'own': 'own',
    'other': 'other',
    'none': 'none',
    'any': 'none'
}
pmt_hist['home_ownership'] = pmt_hist['home_ownership'].str.lower().replace(
    home_ownership_fix)

# public_recs __________________________________________________________________
records_fix = {
    '*': 1
}  #leave nan as nan, but * had at least 1 from m_since_record
pmt_hist['public_recs'] = pmt_hist['public_recs'].replace(records_fix).astype(
    float)

# fico_apply __________________________________________________________________
fico_apply_fix = {'850': '850-850'}
pmt_hist['fico_apply'] = pmt_hist['fico_apply'].replace(fico_apply_fix)
pmt_hist['fico_apply'] = (pmt_hist['fico_apply'].str[:3].astype(int) +
                          pmt_hist['fico_apply'].str[4:].astype(int)) / 2
pmt_hist['fico_apply'] = pmt_hist['fico_apply'].astype(int)

# fico_last ___________________________________________________________________
fico_last_fix = {'845-HIGH': '845-849', 'LOW-499': '495-499'}
pmt_hist['fico_last'] = pmt_hist['fico_last'].replace(fico_last_fix)
pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING', 'fico_last'] = (
    pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING', 'fico_last'].str[:3]
    .astype(int) + pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING',
                               'fico_last'].str[4:].astype(int)) / 2
pmt_hist.ix[pmt_hist['fico_last'] == 'MISSING', 'fico_last'] = pmt_hist.ix[
    pmt_hist['fico_last'] == 'MISSING', 'fico_apply']
pmt_hist['fico_last'] = pmt_hist['fico_last'].astype(int)

# revol_credit_bal ____________________________________________________________
pmt_hist['revol_credit_bal'] = pmt_hist['revol_credit_bal'].astype(
    float)

# fix on a few bad rows where I think there is a mistaken amt_paid ____________
pmt_hist.ix[(pmt_hist['pmt_date'].isnull() & pmt_hist['amt_paid'] > 0),
            'amt_paid'] = 0

# There is a problem with the inquiries 6m column. Some are nan values and some
# are marked '*' with no explanation. inq6m should be in loan info so dropping
pmt_hist.drop('inq_6m', axis=1, inplace=True)

# add a column which is numerical of loan_id for later reading from hdfstore
pmt_hist['loan_id_num'] = pmt_hist['loan_id'].astype('int')

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:98: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [10]:
# save as feather format
PATH = '/home/justin/all_data/lendingclub/'
pmt_hist.to_feather(f'{PATH}pmt_hist_c1.fth')

# Store part 1 of cleaning

In [11]:
# # store pmt_history in hdfstore _______________________________________________
# store = pd.HDFStore(
#     dc.home_path+'/justin_tinkering/data_science/lendingclub/{0}_store.h5'.
#     format(platform),
#     append=True)

# # Create min_itemsize_dict for allocating size when storing ___________________
# min_itemsize_dict = {}
# for col in pmt_hist.columns:
#     if pmt_hist[col].dtype == np.object:
#         print(col, pmt_hist[col].str.len().max())
#         if col in ['State', 'VINTAGE', 'grade']:
#             pass
#         else:
#             min_itemsize_dict[col] = 15

# total_len = len(pmt_hist)
# chunk_size = 120000 # 120k rows at a time
# chunks = np.ceil(total_len/chunk_size)
# df_chunks = np.array_split(pmt_hist, chunks)

     
# k = 0
# for chunk in tqdm_notebook(df_chunks):
#     if k == 0:
#         store.append(
#             'pmt_hist_intermediary_1',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=False,
#             min_itemsize=min_itemsize_dict)
#         k += 1
#     else:
#         store.append(
#             'pmt_hist_intermediary_1',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=True)            
        
# # store pmt_hist ids        
# pmt_hist_ids = pd.Series(pmt_hist['loan_id'].unique())
# pmt_hist_ids.to_hdf(store, 'pmt_hist_ids', mode='w')        
# print(store.keys())
# print(store)        
# store.close()

# Below was investigating to base my choices above on

In [ ]:
# pmt_hist['unaccounted_rec_pmt_money'] = pmt_hist['amt_paid'] - (
#     pmt_hist['princp_paid'] + pmt_hist['int_paid'] + pmt_hist['fee_paid'])

# # Don't care about differences less than a cent _______________________________
# pmt_hist['unaccounted_rec_pmt_money'] = np.where(
#     pmt_hist['unaccounted_rec_pmt_money'] < 0.01, 0,
#     pmt_hist['unaccounted_rec_pmt_money'])

# # These should probably be Received_amt 0 because there is no received_d
# pmt_hist[(pmt_hist['unaccounted_rec_pmt_money'] > 0) & (pmt_hist['PERIOD_END_LSTAT'] != 'Fully Paid')]